In [ ]:
# magics: ensures that any changes to the modules loaded below will be re-loaded automatically
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

# load general packages
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

# load modules related to this exercise
from model_zucher import zurcher
from Solve_NFXP import solve_NFXP
import estimate_NFXP as estimate

#### Before solving the exercise, you should download line_profiler. Line_profiler is a tool to check the performance of our code. To install line_profiler, you can open anaconda prompt and write "pip install line-profiler" (without the " " of course). If you want to know more about line_profiler, check the link below:

https://github.com/rkern/line_profiler.

# Exercise 1

#### Consider the engine replacement model given by:

$$
V(x,\varepsilon) = \max_{d\in \{0,1\}} \big\{ u(x,d) + \varepsilon_d + \beta
\underbrace{\int_{X} \int_{\Omega} V(x',\varepsilon') \pi(x'|x,d) q(\varepsilon'|x') dx' d\varepsilon' }_{EV(x,d)} \big\}
$$

Where $ \varepsilon $ is extreme value Type I distribued and utility is given by:

$$
u(x,d)=\left \{
\begin{array}{ll}
    -RC-c(0,\theta_1) & \text{if }d=\text{replace}=1 \\
    -c(x,\theta_1) & \text{if }d=\text{keep}=0
\end{array} \right.
$$

Here

- $ RC $ = replacement cost  
- $ c(x,\theta_1) $ = cost of maintenance with preference parameters $ \theta_1 $  




#### 1. Look at ReadMe.txt to get an overview of the code

#### 2. Invistigate how the code works, that is ensure you understand:
<il type ="a">
<li> zurcher.init</li>
<li> zurcher.setup</li>
<li> zurcher.create_grid</li>
<li> zucher.state_transition </li>
<li> zucher.bellman </li>

You can see how they are called below
    

#### 3. Run the code below and make sure you understand what we are printing. 
What does the last transition probability in each row (equal to 0.05) come from?

In [ ]:
do_settings = {
    'RC': 0.5,
    'n': 12,
    'p':[0.65,0.2,0.1]   
}
model = zurcher(**do_settings)

print('Model grid:\n',model.grid)
print('Transition probabilities conditional on not replacing:\n',model.P1)
print('Transition probabilities conditional on replacing:\n',model.P2)
ev,pk, dev = model.bellman(np.zeros((model.n)),output=3)
print('Bellman one run:\n',ev)

### Newton's Method

Next, we need to solve the model. Rust 1987 uses Newton–Kantorovich (NK) theorem to solve the Bellman equation in the engine replacement model. To understand the NK algorithm, consider using the Newton's method to solve the single-variable equation, $f(x)=0$. The Newton method uses the iterative procedure stated below to solve the equation:

$$x_{n+1} = x_{n} - \frac{f(x_n)}{f'(x_n)}$$

### 4. Use the Newton's Method to solve the equation below. Fill in the Newton step. Try to vary the starting value and see if the solution changes.


$$f(x) = 3x^2 - exp(x)=0$$

$$f'(x) = g(x) = 6x-exp(x) $$

In [ ]:
f = lambda x: 3*x**2-np.exp(x)
g = lambda x: 6*x-np.exp(x)

def newton(f, g, x0, tol=10e-5, max_iter=100):
    delta = 2000
    it=0
    while (max_iter>= it and tol<delta):
        # fill in
        delta = abs(x1-x0)
        it += 1
        x0 = x1
    return x1, it


x0,it = newton(f = f, g = g, x0 = 5)

x = np.linspace(-2, 4, num=100)
fx = f(x)
plt.plot(x, fx)
plt.axhline(y=0, color='r', linestyle='-')

print('Root of f(x):', round(x0,2))
print('Number of iterations to archieve convergence:', it)

### Newton-Kantorovich

Now consider solving the engine replacement model. To do so, we need to find the expected value function that solves the Bellman equation.

$$
EV(x,d) =  \Gamma(EV)(x,d) \quad\Leftrightarrow\quad (I - \Gamma)(EV)(x,d)=\mathbb{0}
$$

Similar to the Newton iteration, the **NK iteration** uses the following equation

$$
EV_{k+1} = EV_{k} - (I-\Gamma')^{-1} (I-\Gamma)(EV_k)
$$

- The new operator is the difference between the identity operator \$I\$ and Bellman operator $ \Gamma  $  
- $ \mathbb{0} $ is zero function  
- $ I-\Gamma' $ is a Fréchet derivative of the operator $ I-\Gamma $  

### 5. Solve the model. In order to solve the model, you should understand:
<li> solve_NFXP.init</li>
<li> solve_NFXP.setup</li>
<li> solve_NFXP.poly </li>
<li> solve_NFXP.sa </li>
<li> solve_NFXP.nk </li>
</il>
You can see how they are called below: 

In [ ]:
algorithm = 'poly'
do_settings_solver = {
    'sa_min': 10,
    'sa_max': 1000000,  
    'printfxp': 2
}

solver = solve_NFXP(**do_settings_solver)
model = zurcher()

ev0 = np.zeros((model.n)) # Initial guess
if algorithm == 'sa':
    ev = solver.sa(model.bellman, ev0)
elif algorithm == 'poly':
    ev = solver.poly(model.bellman, ev0, beta = model.beta)
else:
    print('Algorithm must be "sa" or "poly"')


#### 6. Now we have to estimate the model. In order to estimate the model, you should understand:
<il type ="a">
<li> zurcher.read_busdata </li>
<li> estimate_NFXP.estimate  </li>
<li> estimate_NFXP.ll  </li>
</il>

You can see how they are called below:

#### 7. Estimate the model

In [ ]:
# Set up the model
model = zurcher()

# Set-up solver
solver = solve_NFXP()

# Read the data
data = model.read_busdata(bustypes=[1,2,3,4])
samplesize = data.shape[0]

# Estimate the model
import time
t0 = time.time()
theta0 = [0,0]

# args for nfxp estimate
nfxp_model, optim_res, pnames, theta_hat, Avar, converged=estimate.estimate(model, solver,data,theta0=theta0, twostep=0)

t1 = time.time()
time = t1-t0

# Print the result
print(f'Structual estimation using busdata from Rust(1987)')
print(f'Beta        = {model.beta:.4f}')
print(f'n           = {model.n}')
print(f'Sample size = {samplesize}\n \n')

print(f'Parameters     Estimates    s.e. ') 
print(f'{pnames[0]}             {theta_hat[0]:.4f}     {np.sqrt(Avar[0,0]):.4f} ')
print(f'{pnames[1]}              {theta_hat[1]:.4f}     {np.sqrt(Avar[1,1]):.4f} \n ')
print(f'{pnames[2]}(1)           {theta_hat[2]:.4f}     {np.sqrt(Avar[2,2]):.4f}  ')
print(f'{pnames[2]}(2)           {theta_hat[3]:.4f}     {np.sqrt(Avar[3,3]):.4f}  ')
print(f'{pnames[2]}(3)           {theta_hat[4]:.4f}     {np.sqrt(Avar[4,4]):.4f}  ')
print(f'{pnames[2]}(4)           {theta_hat[5]:.4f}     {np.sqrt(Avar[5,5]):.4f}  \n')


print(f'Log-likelihood {-optim_res.fun*samplesize:.2f}') 
print(f'runtime (seconds) {time:.4f}')
print(f'The model converged: {converged}')

#### 8. Try using line_profiler in python. This gives you a lot of information about the performance of your code

In [ ]:
%lprun -f estimate.ll  -f estimate.estimate estimate.estimate(model, solver,data,theta0=theta0, twostep=0)

In [ ]:
ev0 = np.zeros((model.n)) # Initial guess
%lprun -f solve_NFXP.nk -f solve_NFXP.poly solve_NFXP.poly(solver,model.bellman, ev0, beta = model.beta)

#### 9. In the code, we are using analytical hessian and gradient. Let us now try to use numerical equivalents. 


a) Now try changing the optimizer options, and turn the use of the non-numerical Hessian off . What happens?

b) Now also try it with the analytical gradient off, what happens?

In [ ]:
import alternative_specifications_ex7 as a_s_ex7
import warnings
# Turn off warnings: We turn of warnings as a result of overflow. This occurs as the optimizer will sometimes guess on non-feasible transition probabilities. 
warnings.filterwarnings("ignore", category=RuntimeWarning)

model = zurcher()
solver = solve_NFXP()

#Ordinaty
print('BHHH:')
%timeit nfxp_results = a_s_ex7.estimate(model, solver,data,theta0=theta0, twostep=0,est_type=0)


# Hessian off
print('')
print('Hessian is off:')
%timeit nfxp_result = a_s_ex7.estimate(model, solver,data, twostep=0,est_type=1)


# #Hessian and gradient ofF 
print('')
print('Hessian and gradient are off:')
%timeit nfxp_results = a_s_ex7.estimate(model, solver,data,theta0=theta0, twostep=0,est_type=2)


#### 9. Try estimate the model for different values of $\beta$. 

(a) Why can we not estimate $\beta$?

(b) When estimating with different $\beta$, do the changes in the estimates of c and/or RC make intuitively sense?

(c) Can you think of some data/variation, which could allow us to identify $\beta$?

In [ ]:
# VARY BETA: 
Nbeta = 4
beta = np.linspace(0.5,0.9999,Nbeta)
log_lik = np.nan + np.zeros((Nbeta,1))
theta_hats =  np.nan + np.zeros((Nbeta,2))

data = model.read_busdata(bustypes=[1,2,3,4])
samplesize = data.shape[0]

print(f'beta     RC     C       log_lik')
for i in range(Nbeta):
    
    # Set up the model
    do_settings = {
    'beta': beta[i]
    }
    model = zurcher(**do_settings)


    # Set-up solver
    solver = solve_NFXP()

    # Estimate the model
    theta0 = [0,0]
    nfxp_model, optim_res, pnames, theta_hat, Avar, converged=estimate.estimate(model, solver,data,theta0=theta0, twostep=0)

    
    theta_hats[i,0] = theta_hat[0]
    theta_hats[i,1] = theta_hat[1]
    log_lik[i]=-optim_res.fun*samplesize
    print(f'{beta[i]:.4f} {theta_hats[i,0]:.4f} {theta_hats[i,1]:.4f} {log_lik[i]} ')



#### 10. We use the latest EV guess to start the solve-procedure even though we change $\theta$ from one likelihood iteration to another. Why do you think we do that? 
(a) What if we started over with EV=0 each iteration? Try that and see what happens with the parameters and the numerical performance.

In [ ]:
import alternative_specifications_ex9 as a_s_ex9 

# Ordinary
print('Same EV')
%timeit a_s_ex9.estimate(model, solver,data,0)
nfxp_results_ord, theta_hat_ord = a_s_ex9.estimate(model, solver,data,0)


# Change EV=0 in each iteration
print('EV=0')
%timeit a_s_ex9.estimate(model, solver,data,1)
nfxp_results_diff, theta_hat_diff = a_s_ex9.estimate(model, solver,data,1)

print('')
print(f'                 Same EV       EV=0')
print(f'{pnames[0]}               {theta_hat_ord[0]:.4f}       {theta_hat_diff[0]:.4f}')
print(f'{pnames[1]}                {theta_hat_ord[1]:.4f}       {theta_hat_diff[1]:.4f}')

#### 11. Try setting the maximum number of miles (odometer reading) to 720(multiplied previous maximum by 1.6). Now the absorbing state is much higher. 

(a) If we adjust the number of grid points as well, so that we have a comparable model (multiply the number of grids by 1.6), do we get a better fit? 

(b) Try to lower the number of grid points to 175 again. How do the parameters change? Are the changes intuitive? 

(c) Optional: What if you change the max to 225 and half the number of grids (hint: what goes wrong?)?

In [ ]:
# Function for adjusting Grid-points
def adjust_grid_point(maks, n):
    # Set up the model
    do_settings = {
    'max': maks,
    'n': n
    }
    model = zurcher(**do_settings)

    # Set-up solver
    solver = solve_NFXP()
        
    # Read the data
    data = model.read_busdata(bustypes=[1,2,3,4])
    samplesize = data.shape[0]

    # Estimate the model
    theta0 = [0,0]
    
    nfxp_model, result, pnames, theta, Avar, converged=estimate.estimate(model, solver,data,theta0=theta0, twostep=0)

    
    print(f'Parameters     Estimates    s.e. ') 
    print(f'{pnames[0]}             {theta[0]:.4f}     {np.sqrt(Avar[0,0]):.4f} ')
    print(f'{pnames[1]}              {theta[1]:.4f}     {np.sqrt(Avar[1,1]):.4f} \n ')
    print(f'Log-likelihood now {-result.fun*samplesize:.4f}\n \n') 


In [ ]:
# Baseline max = 450, n = 175
print(f'Baseline')
adjust_grid_point(450,175)

# a)  max = 720, n = 280
print(f'Question (a)')
adjust_grid_point(int(450*1.6),int(175*1.6))

# b) max = 720., n = 175
print(f'Question (b)')
adjust_grid_point(int(450*1.6),175)

# c) max =225, n = 175/2
print(f'Question (c)')
adjust_grid_point(int(450/2),int(175/2));